# Introduction

In this notebook, I will talk about how to use Auto-Keras, an open source alternative to Google’s AutoML, for automated machine learning and deep learning.

When training a neural network on a dataset there are two primary objectives a deep learning practitioner is trying to optimize and balance:

* Defining a neural network architecture that lends itself to the nature of the dataset
* Tuning a set of hyperparameters over many experiments that will lead to a model with high accuracy and ability to generalize to data outside the training and testing sets. 

Typical hyperparameters that need to be tuned include the optimizer algorithm (SGD, Adam, etc.), learning rate and learning rate scheduling, and regularization, etc.

Depending on the dataset and problem, it can take a deep learning expert upwards of tens to hundreds of experiments to find a balance between neural network architecture and hyperparameters.

The end goal of both __Auto-Keras__ and __Google AutoML__ is to reduce the barrier to entry to performing machine learning and deep learning through the use of automated __Neural Architecture Search (NAS)__ algorithms. This type of work on automatically train and tune a neural network is usually called __Automated Machine Learning (AutoML)__.

## Automated Machine Learning (AutoML)

AutoML enables developers and engineers to automatically train neural networks on their own datasets with following iterative steps:

* Training a network on a training set
* Evaluating the network on a testing set
* Modifying the neural network architecture
* Tuning hyperparameters
* Repeating the process

The programmer or engineer using AutoML doesn’t need to define their own neural network architecture or tune the hyperparameters — AutoML is doing that for them automatically. Some people include more components in the definition of Automated Machine Learning, such as:

* __Automated feature engineering__
* __Automated model selection and hyperparameter tuning__
* __Automated neural network architecture selection__
* __Automated deployment__

The most challenging tasks in an AutoML system are __automated neural network architecture selection__ and __automated model selection and hyperparameter tuning__.

The following diagram shows the relationship among AutoML, hypterparameter Optimization and Neural Architecture Search.
![automl_nas](./automl_nas.png)

## Neural Architecture Search (NAS)

Given the input dataset, a Neural Architecture Search algorithm will automatically search for the most optimal architecture and corresponding parameters.

For example, in the context of computer vision and image recognition, a Neural Architecture Search algorithm will:

* Accept an input training dataset
* Optimize and find architectural building blocks called “cells” — these cells are automatically learned and may look similar to inception, residual, or squeeze/fire micro-architectures
* Continually train and search the “NAS search space” for more optimized cells

If the user of the AutoML system is an experienced deep learning practitioner then they may decide to:

* Run the NAS on a significantly smaller subset of the training dataset
* Find an optimal set of architectural building blocks/cells
* Take these cells and manually define a deeper version of the network found during the architecture search
* Train the network on the full training set using their own expertise and best practices

Such an approach is a hybrid between a fully automated machine learning solution and one that requires an expert deep learning practitioner — often this approach will lead to better accuracy than what the NAS finds on its own. The following diagram shows the key components involved in NAS:
![nas_components](./nas_components.png)

Instead of designing complex deep networks, one can just run a preset NAS algorithm. Google recently took this to the extreme by offering Cloud AutoML. Just upload your data and Google’s NAS algorithm will find you an architecture, quick and easy! Cloud AutoML does have a steep price of $20 USD and unfortunately you can’t export your model once it’s trained; you’ll have to use their API to run your network on the cloud.

# Auto-Keras

[Auto-Keras](https://autokeras.com/),  developed by the DATA Lab team at Texas A&M University, is considered as a very popular open source alternative to Google’s AutoML. It doesn't only utilizes the Neural Architecture Search but applies “network morphism” (keeping network functionality while changing the architecture) along with Bayesian optimization to guide the network morphism for more efficient neural network search.

## Auto-Keras Github

![autokeras_github](./autokeras_github.PNG)

For some details or getting deeper understanding on AutoML and NAS, recommend to further read articles listed in the "References" section.

## Installation of AutoKeras

Note: Currently, AutoKeras 1.0.2 is only compatible with Python >= 3.5 and TensorFlow >= 2.1.0.

# Text Classification

In this sections, I will demonstrate how to use Auto-Keras to automatically train a text classifier model with the data set of IMDB.

## Python Libraries

In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
import autokeras as ak

# only logging error information
import logging
logging.getLogger().setLevel(logging.ERROR)
# disable warning message show-ip
import warnings
warnings.filterwarnings("ignore")

# Allow multiple output/display from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load Data Set

In [2]:
# Load the integer sequence the IMDB dataset with Keras.
index_offset = 3  # word index offset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000, index_from=index_offset)
# display the top 5 training data
x_train[0:3]
y_train[0:3]

array([list([1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 134, 26, 4, 715, 8, 118, 2, 14, 394, 20, 13, 119, 954, 189,

array([1, 0, 0], dtype=int64)

## Prepare Training Data

In [3]:
# find all word and word index mapping
word_to_id = imdb.get_word_index()
word_to_id = {k: (v + index_offset) for k, v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
# Prepare the dictionary of index to word.
id_to_word = {value: key for key, value in word_to_id.items()}
# display the mappings
id_to_word

{34704: 'fawn',
 52009: 'tsukino',
 52010: 'nunnery',
 16819: 'sonja',
 63954: 'vani',
 1411: 'woods',
 16118: 'spiders',
 2348: 'hanging',
 2292: 'woody',
 52011: 'trawling',
 52012: "hold's",
 11310: 'comically',
 40833: 'localized',
 30571: 'disobeying',
 52013: "'royale",
 40834: "harpo's",
 52014: 'canet',
 19316: 'aileen',
 52015: 'acurately',
 52016: "diplomat's",
 25245: 'rickman',
 6749: 'arranged',
 52017: 'rumbustious',
 52018: 'familiarness',
 52019: "spider'",
 68807: 'hahahah',
 52020: "wood'",
 40836: 'transvestism',
 34705: "hangin'",
 2341: 'bringing',
 40837: 'seamier',
 34706: 'wooded',
 52021: 'bravora',
 16820: 'grueling',
 1639: 'wooden',
 16821: 'wednesday',
 52022: "'prix",
 34707: 'altagracia',
 52023: 'circuitry',
 11588: 'crotch',
 57769: 'busybody',
 52024: "tart'n'tangy",
 14132: 'burgade',
 52026: 'thrace',
 11041: "tom's",
 52028: 'snuggles',
 29117: 'francesco',
 52030: 'complainers',
 52128: 'templarios',
 40838: '272',
 52031: '273',
 52133: 'zaniacs',

In [4]:
# Convert the word indices to words.
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
x_train = list(map(lambda sentence: ' '.join(id_to_word[i] for i in sentence), x_train))
x_test = list(map(lambda sentence: ' '.join(id_to_word[i] for i in sentence), x_test))
x_train = np.array(x_train, dtype=np.str)
x_test = np.array(x_test, dtype=np.str)
print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)

(25000,)
(25000, 1)


In [5]:
# display the first record
print(x_train[0])

<START> this film was just brilliant casting <UNK> <UNK> story direction <UNK> really <UNK> the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same <UNK> <UNK> as myself so i loved the fact there was a real <UNK> with this film the <UNK> <UNK> throughout the film were great it was just brilliant so much that i <UNK> the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the <UNK> <UNK> was amazing really <UNK> at the end it was so sad and you know what they say if you <UNK> at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of <UNK> and paul they were just brilliant children are often left out of the <UNK> <UNK> i think because the stars that play them all <UNK> up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they have done don't yo

## Find the Best Model with Auto-Keras

In [9]:
# Initialize the text classifier.
# It tries 2 different models (long time to run).
clf = ak.TextClassifier(max_trials=2)
# Feed the text classifier with training data.
clf.fit(x_train, y_train, validation_split=0.15, epochs=10) # Split the training data and use the last 15% as validation data.

Train for 665 steps, validate for 118 steps
Epoch 1/10


184/665 [=======>......................] - ETA: 10:10 - loss: 0.6884 - accuracy: 0.593 - ETA: 5:55 - loss: 0.6823 - accuracy: 0.609 - ETA: 4:25 - loss: 0.6819 - accuracy: 0.59 - ETA: 3:40 - loss: 0.6833 - accuracy: 0.58 - ETA: 3:11 - loss: 0.6801 - accuracy: 0.58 - ETA: 2:54 - loss: 0.6851 - accuracy: 0.57 - ETA: 2:43 - loss: 0.6862 - accuracy: 0.57 - ETA: 2:33 - loss: 0.6959 - accuracy: 0.54 - ETA: 2:25 - loss: 0.7027 - accuracy: 0.52 - ETA: 2:19 - loss: 0.7011 - accuracy: 0.53 - ETA: 2:14 - loss: 0.7035 - accuracy: 0.51 - ETA: 2:11 - loss: 0.7038 - accuracy: 0.50 - ETA: 2:07 - loss: 0.7029 - accuracy: 0.50 - ETA: 2:04 - loss: 0.7020 - accuracy: 0.50 - ETA: 2:02 - loss: 0.6993 - accuracy: 0.51 - ETA: 2:00 - loss: 0.7004 - accuracy: 0.51 - ETA: 1:57 - loss: 0.7002 - accuracy: 0.51 - ETA: 1:56 - loss: 0.6997 - accuracy: 0.51 - ETA: 1:54 - loss: 0.6978 - accuracy: 0.52 - ETA: 1:52 - loss: 0.6983 - accuracy: 0.51 - ETA: 1:51 - loss: 0.6993 - accuracy: 0.51 - ETA: 1:50 - loss: 0.6989 - acc

665/665 [==============================] - ETA: 15s - loss: 0.4935 - accuracy: 0.740 - ETA: 14s - loss: 0.4933 - accuracy: 0.741 - ETA: 14s - loss: 0.4929 - accuracy: 0.741 - ETA: 14s - loss: 0.4925 - accuracy: 0.741 - ETA: 14s - loss: 0.4923 - accuracy: 0.741 - ETA: 14s - loss: 0.4922 - accuracy: 0.741 - ETA: 14s - loss: 0.4921 - accuracy: 0.742 - ETA: 14s - loss: 0.4919 - accuracy: 0.742 - ETA: 13s - loss: 0.4913 - accuracy: 0.742 - ETA: 13s - loss: 0.4909 - accuracy: 0.742 - ETA: 13s - loss: 0.4904 - accuracy: 0.743 - ETA: 13s - loss: 0.4901 - accuracy: 0.743 - ETA: 13s - loss: 0.4899 - accuracy: 0.743 - ETA: 13s - loss: 0.4895 - accuracy: 0.744 - ETA: 13s - loss: 0.4893 - accuracy: 0.744 - ETA: 12s - loss: 0.4891 - accuracy: 0.744 - ETA: 12s - loss: 0.4890 - accuracy: 0.744 - ETA: 12s - loss: 0.4886 - accuracy: 0.744 - ETA: 12s - loss: 0.4886 - accuracy: 0.744 - ETA: 12s - loss: 0.4884 - accuracy: 0.744 - ETA: 12s - loss: 0.4884 - accuracy: 0.745 - ETA: 12s - loss: 0.4881 - accurac

186/665 [=======>......................] - ETA: 1:30 - loss: 0.2428 - accuracy: 0.87 - ETA: 1:24 - loss: 0.2841 - accuracy: 0.84 - ETA: 1:25 - loss: 0.2811 - accuracy: 0.84 - ETA: 1:28 - loss: 0.3027 - accuracy: 0.83 - ETA: 1:28 - loss: 0.2993 - accuracy: 0.85 - ETA: 1:28 - loss: 0.2941 - accuracy: 0.86 - ETA: 1:28 - loss: 0.2969 - accuracy: 0.85 - ETA: 1:27 - loss: 0.2916 - accuracy: 0.85 - ETA: 1:29 - loss: 0.2822 - accuracy: 0.86 - ETA: 1:28 - loss: 0.3051 - accuracy: 0.85 - ETA: 1:27 - loss: 0.3367 - accuracy: 0.84 - ETA: 1:28 - loss: 0.3453 - accuracy: 0.83 - ETA: 1:28 - loss: 0.3366 - accuracy: 0.84 - ETA: 1:28 - loss: 0.3337 - accuracy: 0.84 - ETA: 1:28 - loss: 0.3419 - accuracy: 0.84 - ETA: 1:28 - loss: 0.3418 - accuracy: 0.84 - ETA: 1:28 - loss: 0.3484 - accuracy: 0.84 - ETA: 1:28 - loss: 0.3471 - accuracy: 0.84 - ETA: 1:27 - loss: 0.3463 - accuracy: 0.84 - ETA: 1:27 - loss: 0.3541 - accuracy: 0.83 - ETA: 1:29 - loss: 0.3524 - accuracy: 0.83 - ETA: 1:29 - loss: 0.3592 - accura

665/665 [==============================] - ETA: 14s - loss: 0.3157 - accuracy: 0.867 - ETA: 14s - loss: 0.3154 - accuracy: 0.867 - ETA: 14s - loss: 0.3153 - accuracy: 0.867 - ETA: 14s - loss: 0.3156 - accuracy: 0.867 - ETA: 13s - loss: 0.3158 - accuracy: 0.867 - ETA: 13s - loss: 0.3156 - accuracy: 0.867 - ETA: 13s - loss: 0.3152 - accuracy: 0.867 - ETA: 13s - loss: 0.3149 - accuracy: 0.867 - ETA: 13s - loss: 0.3146 - accuracy: 0.867 - ETA: 13s - loss: 0.3144 - accuracy: 0.868 - ETA: 13s - loss: 0.3143 - accuracy: 0.868 - ETA: 13s - loss: 0.3142 - accuracy: 0.868 - ETA: 12s - loss: 0.3140 - accuracy: 0.868 - ETA: 12s - loss: 0.3143 - accuracy: 0.868 - ETA: 12s - loss: 0.3142 - accuracy: 0.868 - ETA: 12s - loss: 0.3140 - accuracy: 0.868 - ETA: 12s - loss: 0.3138 - accuracy: 0.868 - ETA: 12s - loss: 0.3137 - accuracy: 0.868 - ETA: 12s - loss: 0.3140 - accuracy: 0.868 - ETA: 11s - loss: 0.3137 - accuracy: 0.868 - ETA: 11s - loss: 0.3136 - accuracy: 0.868 - ETA: 11s - loss: 0.3137 - accurac

...

746/783 [===========================>..] - ETA: 31s - loss: 0.1034 - accuracy: 0.958 - ETA: 31s - loss: 0.1033 - accuracy: 0.958 - ETA: 31s - loss: 0.1032 - accuracy: 0.958 - ETA: 31s - loss: 0.1032 - accuracy: 0.958 - ETA: 31s - loss: 0.1033 - accuracy: 0.958 - ETA: 31s - loss: 0.1032 - accuracy: 0.958 - ETA: 30s - loss: 0.1031 - accuracy: 0.958 - ETA: 30s - loss: 0.1029 - accuracy: 0.958 - ETA: 30s - loss: 0.1030 - accuracy: 0.958 - ETA: 30s - loss: 0.1029 - accuracy: 0.959 - ETA: 30s - loss: 0.1029 - accuracy: 0.959 - ETA: 30s - loss: 0.1028 - accuracy: 0.959 - ETA: 30s - loss: 0.1027 - accuracy: 0.959 - ETA: 29s - loss: 0.1028 - accuracy: 0.959 - ETA: 29s - loss: 0.1030 - accuracy: 0.958 - ETA: 29s - loss: 0.1029 - accuracy: 0.959 - ETA: 29s - loss: 0.1029 - accuracy: 0.958 - ETA: 29s - loss: 0.1029 - accuracy: 0.958 - ETA: 29s - loss: 0.1029 - accuracy: 0.958 - ETA: 29s - loss: 0.1028 - accuracy: 0.958 - ETA: 28s - loss: 0.1030 - accuracy: 0.958 - ETA: 28s - loss: 0.1033 - accurac

783/783 [==============================] - ETA: 5s - loss: 0.1065 - accuracy: 0.95 - ETA: 4s - loss: 0.1064 - accuracy: 0.95 - ETA: 4s - loss: 0.1064 - accuracy: 0.95 - ETA: 4s - loss: 0.1063 - accuracy: 0.95 - ETA: 4s - loss: 0.1063 - accuracy: 0.95 - ETA: 4s - loss: 0.1063 - accuracy: 0.95 - ETA: 4s - loss: 0.1063 - accuracy: 0.95 - ETA: 4s - loss: 0.1063 - accuracy: 0.95 - ETA: 3s - loss: 0.1064 - accuracy: 0.95 - ETA: 3s - loss: 0.1064 - accuracy: 0.95 - ETA: 3s - loss: 0.1065 - accuracy: 0.95 - ETA: 3s - loss: 0.1064 - accuracy: 0.95 - ETA: 3s - loss: 0.1064 - accuracy: 0.95 - ETA: 3s - loss: 0.1065 - accuracy: 0.95 - ETA: 3s - loss: 0.1065 - accuracy: 0.95 - ETA: 2s - loss: 0.1065 - accuracy: 0.95 - ETA: 2s - loss: 0.1066 - accuracy: 0.95 - ETA: 2s - loss: 0.1066 - accuracy: 0.95 - ETA: 2s - loss: 0.1065 - accuracy: 0.95 - ETA: 2s - loss: 0.1066 - accuracy: 0.95 - ETA: 2s - loss: 0.1065 - accuracy: 0.95 - ETA: 2s - loss: 0.1065 - accuracy: 0.95 - ETA: 1s - loss: 0.1064 - accuracy

186/783 [======>.......................] - ETA: 2:03 - loss: 0.0969 - accuracy: 0.93 - ETA: 2:01 - loss: 0.1211 - accuracy: 0.93 - ETA: 1:56 - loss: 0.1014 - accuracy: 0.95 - ETA: 1:56 - loss: 0.1181 - accuracy: 0.94 - ETA: 1:53 - loss: 0.1031 - accuracy: 0.95 - ETA: 1:53 - loss: 0.0945 - accuracy: 0.95 - ETA: 1:53 - loss: 0.0879 - accuracy: 0.96 - ETA: 1:52 - loss: 0.0832 - accuracy: 0.96 - ETA: 1:51 - loss: 0.0791 - accuracy: 0.96 - ETA: 1:50 - loss: 0.0818 - accuracy: 0.96 - ETA: 1:50 - loss: 0.1013 - accuracy: 0.96 - ETA: 1:51 - loss: 0.1058 - accuracy: 0.95 - ETA: 1:50 - loss: 0.1002 - accuracy: 0.95 - ETA: 1:49 - loss: 0.0958 - accuracy: 0.96 - ETA: 1:49 - loss: 0.0920 - accuracy: 0.96 - ETA: 1:49 - loss: 0.0898 - accuracy: 0.96 - ETA: 1:49 - loss: 0.0927 - accuracy: 0.96 - ETA: 1:49 - loss: 0.0997 - accuracy: 0.96 - ETA: 1:49 - loss: 0.0988 - accuracy: 0.96 - ETA: 1:48 - loss: 0.0982 - accuracy: 0.96 - ETA: 1:48 - loss: 0.0966 - accuracy: 0.96 - ETA: 1:48 - loss: 0.0940 - accura

747/783 [===========================>..] - ETA: 31s - loss: 0.0967 - accuracy: 0.962 - ETA: 31s - loss: 0.0966 - accuracy: 0.963 - ETA: 31s - loss: 0.0969 - accuracy: 0.963 - ETA: 31s - loss: 0.0970 - accuracy: 0.962 - ETA: 31s - loss: 0.0974 - accuracy: 0.962 - ETA: 30s - loss: 0.0974 - accuracy: 0.962 - ETA: 30s - loss: 0.0974 - accuracy: 0.962 - ETA: 30s - loss: 0.0973 - accuracy: 0.962 - ETA: 30s - loss: 0.0972 - accuracy: 0.962 - ETA: 30s - loss: 0.0971 - accuracy: 0.962 - ETA: 30s - loss: 0.0973 - accuracy: 0.962 - ETA: 30s - loss: 0.0974 - accuracy: 0.962 - ETA: 29s - loss: 0.0975 - accuracy: 0.962 - ETA: 29s - loss: 0.0979 - accuracy: 0.962 - ETA: 29s - loss: 0.0981 - accuracy: 0.962 - ETA: 29s - loss: 0.0979 - accuracy: 0.962 - ETA: 29s - loss: 0.0981 - accuracy: 0.962 - ETA: 29s - loss: 0.0980 - accuracy: 0.962 - ETA: 29s - loss: 0.0979 - accuracy: 0.962 - ETA: 28s - loss: 0.0978 - accuracy: 0.962 - ETA: 28s - loss: 0.0980 - accuracy: 0.962 - ETA: 28s - loss: 0.0979 - accurac

783/783 [==============================] - ETA: 4s - loss: 0.1041 - accuracy: 0.96 - ETA: 4s - loss: 0.1041 - accuracy: 0.96 - ETA: 4s - loss: 0.1041 - accuracy: 0.96 - ETA: 4s - loss: 0.1041 - accuracy: 0.96 - ETA: 4s - loss: 0.1040 - accuracy: 0.96 - ETA: 4s - loss: 0.1040 - accuracy: 0.96 - ETA: 4s - loss: 0.1039 - accuracy: 0.96 - ETA: 3s - loss: 0.1039 - accuracy: 0.96 - ETA: 3s - loss: 0.1039 - accuracy: 0.96 - ETA: 3s - loss: 0.1040 - accuracy: 0.96 - ETA: 3s - loss: 0.1039 - accuracy: 0.96 - ETA: 3s - loss: 0.1038 - accuracy: 0.96 - ETA: 3s - loss: 0.1038 - accuracy: 0.96 - ETA: 3s - loss: 0.1038 - accuracy: 0.96 - ETA: 2s - loss: 0.1038 - accuracy: 0.96 - ETA: 2s - loss: 0.1038 - accuracy: 0.96 - ETA: 2s - loss: 0.1038 - accuracy: 0.96 - ETA: 2s - loss: 0.1038 - accuracy: 0.96 - ETA: 2s - loss: 0.1038 - accuracy: 0.96 - ETA: 2s - loss: 0.1037 - accuracy: 0.96 - ETA: 2s - loss: 0.1039 - accuracy: 0.96 - ETA: 1s - loss: 0.1040 - accuracy: 0.96 - ETA: 1s - loss: 0.1041 - accuracy

## Evaluate the Model with Testing Data

In [10]:
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

350/782 [============>.................] - ETA: 3:10 - loss: 0.6607 - accuracy: 0.81 - ETA: 1:19 - loss: 0.3834 - accuracy: 0.89 - ETA: 57s - loss: 0.4141 - accuracy: 0.8687 - ETA: 49s - loss: 0.3986 - accuracy: 0.870 - ETA: 45s - loss: 0.3715 - accuracy: 0.878 - ETA: 42s - loss: 0.4205 - accuracy: 0.863 - ETA: 40s - loss: 0.4134 - accuracy: 0.867 - ETA: 39s - loss: 0.3908 - accuracy: 0.870 - ETA: 37s - loss: 0.3736 - accuracy: 0.875 - ETA: 36s - loss: 0.3603 - accuracy: 0.881 - ETA: 35s - loss: 0.3619 - accuracy: 0.885 - ETA: 34s - loss: 0.3433 - accuracy: 0.889 - ETA: 33s - loss: 0.3486 - accuracy: 0.883 - ETA: 33s - loss: 0.3414 - accuracy: 0.885 - ETA: 32s - loss: 0.3333 - accuracy: 0.886 - ETA: 32s - loss: 0.3531 - accuracy: 0.883 - ETA: 32s - loss: 0.3531 - accuracy: 0.884 - ETA: 31s - loss: 0.3571 - accuracy: 0.881 - ETA: 31s - loss: 0.3542 - accuracy: 0.880 - ETA: 31s - loss: 0.3609 - accuracy: 0.877 - ETA: 31s - loss: 0.3595 - accuracy: 0.880 - ETA: 30s - loss: 0.3646 - accura

# Regression

In this sections, I will demonstrate how to use Auto-Keras to automatically train a regression model with the data set of auto-mpg.

## Python Libraries

In [68]:
import tensorflow as tf
import pandas as pd
import autokeras as ak

## Load Data Set

In [69]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
dataset_path = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset = dataset.dropna()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


## Prepare Training Data

In [70]:
data_cols = dataset.columns[1:]
data_type = (len(data_cols)-1) * ['numerical'] + ['categorical']
data_type = dict(zip(data_cols, data_type))

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_dataset.describe()
train_dataset.tail()
test_dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,23.310510,5.477707,195.318471,104.869427,2990.251592,15.559236,75.898089,1.573248
std,7.728652,1.699788,104.331589,38.096214,843.898596,2.789230,3.675642,0.800988
min,10.000000,3.000000,68.000000,46.000000,1649.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.500000,76.250000,2256.500000,13.800000,73.000000,1.000000
50%,22.000000,4.000000,151.000000,94.500000,2822.500000,15.500000,76.000000,1.000000
75%,28.950000,8.000000,265.750000,128.000000,3608.000000,17.200000,79.000000,2.000000
max,46.600000,8.000000,455.000000,225.000000,5140.000000,24.800000,82.000000,3.000000


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
281,19.8,6,200.0,85.0,2990.0,18.2,79,1
229,16.0,8,400.0,180.0,4220.0,11.1,77,1
150,26.0,4,108.0,93.0,2391.0,15.5,74,3
145,32.0,4,83.0,61.0,2003.0,19.0,74,3
182,28.0,4,107.0,86.0,2464.0,15.5,76,2


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
369,34.0,4,112.0,88.0,2395.0,18.0,82,1
375,36.0,4,105.0,74.0,1980.0,15.3,82,2
382,34.0,4,108.0,70.0,2245.0,16.9,82,3
384,32.0,4,91.0,67.0,1965.0,15.7,82,3
396,28.0,4,120.0,79.0,2625.0,18.6,82,1


## Find the Best Model with Auto-Keras

In [82]:
regressor = ak.StructuredDataRegressor(max_trials=50, column_names=None, column_types=data_type)
regressor.fit(x=train_dataset.drop(columns=['MPG']), y=train_dataset['MPG'], epochs=10, verbose=False)

INFO:tensorflow:Oracle triggered exit


## Evaluate the Model with Testing Data

In [84]:
# Evaluate the accuracy of the found model.
print('Accuracy: {accuracy}'.format(accuracy=regressor.evaluate(x=test_dataset.drop(columns=['MPG']), y=test_dataset['MPG'])))

3/3 [==============================] - ETA: 0s - loss: 38.7378 - mean_squared_error: 38.73 - 0s 39ms/step - loss: 92.9843 - mean_squared_error: 80.2215
Accuracy: [92.98428599039714, 80.22147]


# Issues and Bugs

![bugs](./autokeras_bugs.PNG)

In [85]:
# for example, saving a trained model in windows environment
km = regressor.export_model()
km.save('my_model.h5')

AttributeError: 'TrackableWeightHandler' object has no attribute 'numpy'

# Future of NAS and AutoML

A lot of great work have been made over the past few years in automating deep learning. It makes it more accessible to users and business; the power of deep learning becomes more accessible to the public in general. But, there’s still lots of room to improve.

Architecture search has become far more efficient, however, it still takes a day or days of training with a single GPU. The search space is still really quite limited. The current NAS algorithms still use the structures and building blocks that were hand designed, they just put them together differently!

A potential future direction would be a far wider ranging search. Such algorithms may reveal even more hidden deep learning secrets within these huge and complex networks. Such a search space requires definitely some efficient algorithm designs.

# Summary

The end goal of both Auto-Keras and AutoML is to reduce the barrier to entry to performing machine learning and deep learning through the use of Neural Architecture Search (NAS) algorithms.

The primary benefits include:

* Being able to perform machine learning and deep learning with little expertise
* Obtaining a high accuracy model with the ability to generalize to data outside the training and testing set
* Getting up and running quickly with either a GUI interface or a simple API
* A potentially state-of-the-art performance with little effort

Both Google’s AutoML and Auto-Keras are great steps forward; however, automated machine learning is nowhere near solved. Automatic machine learning (currently) does not beat having expertise in deep learning. The domain expertise, specifically in the data you are working with, is still absolutely critical to obtain a higher accuracy model. There is still quite a bit of work to be done in this area.

Actually, a hybrid approach between a fully automated machine learning solution and one that requires an expert deep learning practitioner will often lead to better accuracy and less human effort on model tuning.

# References

1. [ Auto-Keras: Efficient Neural Architecture Search with Network Morphism](https://arxiv.org/abs/1806.10282)
1. [Auto-Keras and AutoML: A Getting Started Guide](https://www.pyimagesearch.com/2019/01/07/auto-keras-and-automl-a-getting-started-guide/)
1. [Auto-Keras Official Website](https://autokeras.com/)
1. [Auto-Keras Github Website](https://github.com/keras-team/autokeras)
1. [Neural Architecture Search with Reinforcement Learning (Zoph and Le, 2016)](https://arxiv.org/abs/1611.01578)
1. [Learning Transferable Architectures for Scalable Image Recognition (Zoph et al., 2017)](https://arxiv.org/abs/1707.07012)
1. [Automated Machine Learning-Overview](https://medium.com/thinkgradient/automated-machine-learning-an-overview-5a3595d5c4b5)
1. [Auto-SKLearn (for Linux Only)](https://automl.github.io/auto-sklearn/master/index.html)